# Samplers veya Schedulers

Sampler, bir görüntüye yinelemeli olarak gürültü eklemek veya model çıktısına dayalı olarak bir örneği güncellemek için kullanılır.

* Bu tutorial'da aşağıdaki smapler örnekleri incelenecek ve farklarına bakılacak:

    1) ddim
    2) ddpm
    3) eular
    4) euler_ancestral
    5) pndm
    6) stochastic_karras_ve
    7) ipndm

In [ ]:
import torch
import numpy as np
import PIL

from diffusers import DiffusionPipeline, StableDiffusionImg2ImgPipeline, AutoPipelineForImage2Image
from diffusers import DDIMScheduler
from diffusers import DDIMInverseScheduler
from diffusers import DEISMultistepScheduler
from diffusers import KDPM2AncestralDiscreteScheduler
from diffusers import KDPM2DiscreteScheduler

from diffusers import DDPMScheduler
from diffusers import EulerDiscreteScheduler
from diffusers import EulerAncestralDiscreteScheduler
from diffusers import PNDMScheduler
from diffusers import HeunDiscreteScheduler

import matplotlib.pyplot as plt

In [ ]:
prompt = "a beautiful cat"
steps = 50
strength = 0.6
guidance_scale = 7
generator = torch.Generator(device='cuda').manual_seed(8)
adapter_id = "latent-consistency/lcm-lora-sdv1-5"

input = PIL.Image.open("../../1_media/input_images/cat.jpg")

In [ ]:
pipe = AutoPipelineForImage2Image.from_pretrained("runwayml/stable-diffusion-v1-5", safety_checker=None)
pipe.to("cuda", torch.float16)
print(pipe.scheduler)
pipe.load_lora_weights(adapter_id)
pipe.fuse_lora()

In [ ]:
sch_list = [
    DDIMScheduler,
    DEISMultistepScheduler,
    KDPM2AncestralDiscreteScheduler,
    KDPM2DiscreteScheduler,
    DDPMScheduler,
    EulerDiscreteScheduler,
    EulerAncestralDiscreteScheduler,
    PNDMScheduler,
    HeunDiscreteScheduler
]

In [ ]:
outputs = []
outputs.append({"name": "input", "image": input})
for sch in sch_list:
    pipe.scheduler = sch.from_config(pipe.scheduler.config)
    generator = torch.Generator(device='cuda').manual_seed(42)
    image = pipe(image = input, prompt=prompt, num_inference_steps=steps, generator=generator, strength=strength, guidance_scale=guidance_scale).images[0]
    print(sch.__name__)
    outputs.append({"name": sch.__name__, "image": image})
    

In [ ]:
#matplotlib grid
fig, axs = plt.subplots(5, 2, figsize=(15, 40))
for i, ax in enumerate(axs.flat):
    ax.imshow(outputs[i]["image"])
    ax.set_title(outputs[i]["name"])
    # info text
    ax.axis('off')
ax.text(0.5,-0.5, f"steps: {steps}\nstrength: {strength}\nguidance_scale: {guidance_scale}\nprompt: {prompt}", size=12, ha="center", transform=ax.transAxes)